In [11]:
# Imports
import g4f
import tiktoken
import json
import pandas as pd

In [2]:
# Start settings
system_message = [{"role": "system", "content": "Ты пират, отвечай как пират."}]
max_response_tokens = 1024
token_limit = 16000
conversation = system_message.copy()

In [3]:
# Update system prompt
conversation[0]["content"] = "Ты пират, отвечай как пират."

In [22]:
# Reading prompts from csv files
prompts = pd.read_csv('C:/Users/KorolOrol/Desktop/TUSUR/repos/OPDDungeonMaster/Приключения/Приключения.csv')
conversation[0]["content"] = prompts['prompt'][0]

In [4]:
# Count tokens
def num_tokens_from_messages(messages):
    encoding= tiktoken.get_encoding("cl100k_base")  #model to encoding mapping https://github.com/openai/tiktoken/blob/main/tiktoken/model.py
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

In [9]:
# Chat response
def response(message):
    if message != "":
        conversation.append({"role": "user", "content": message})

    # Remove old messages if token limit is reached
    conv_history_tokens = num_tokens_from_messages(conversation)
    while conv_history_tokens + max_response_tokens >= token_limit:
        del conversation[1] 
        conv_history_tokens = num_tokens_from_messages(conversation)

    answer = g4f.ChatCompletion.create(
        model=g4f.models.gpt_35_turbo_16k_0613,
        provider=g4f.Provider.NeuroGPT,
        messages=conversation,
        stream=True,
    )

    full_message = ""
    for message in answer:
        full_message += message
        print(message, end="", flush=True)
    if full_message == "":
        del conversation[-1]
        response(message)
        return
    print()
    conversation.append({"role": "assistant", "content": full_message})

In [6]:
# Journal realization
journal = {"items": [],
           "characters": [],
           "places": [],
           "actions": []}

In [7]:
# Analyze message
def analyze(message):
    answer = g4f.ChatCompletion.create(
        model=g4f.models.gpt_35_turbo_16k_0613,
        provider=g4f.Provider.NeuroGPT,
        messages=[{"role": "system", "content": "Проанализируй данный текст и найди в нем предмет. Если здесь нет предмета, то напиши 'None'. Если предмет найден, то напиши в формате \{\"name\": \"Название предмета\", \"description\": \"Описание предмета\"\}"},
                  {"role": "user", "content": message}],
        stream=False,
    )
    answer = json.loads(answer)
    return answer

In [23]:
while True:
    message = input()
    if message == "Выход":
        break
    elif message == "Анализ":
        journal["items"].append(analyze(conversation[-1]["content"]))
    elif message == "Чево":
        del conversation[-1]
        response("")
    elif message == "Журнал":
        for category, notes in journal.items():
            print(f"{category}: ")
            for note in notes:
                for key, value in note.items():
                    print(f"    {key}: {value}")
    elif message == "Сначала":
        conversation = system_message.copy()
        journal = {"items": [],
                   "characters": [],
                   "places": [],
                   "actions": []}
    elif message == "Диалог":
        for message in conversation:
            for key, value in message.items():
                print(f"{key}: {value}")
    else:
        response(message)

Привет! Добро пожаловать в мой мир фэнтези и игру Dungeon & Dragons. Я рад, что ты здесь. Давай начнем с создания твоего персонажа. Как тебя зовут?

(игрок может выбрать имя для своего персонажа и сообщить его)

Отлично, очень приятно познакомиться, [Имя игрока]! Теперь давай определимся с расой твоего персонажа. В моем мире есть несколько опций: люди, эльфы, гномы, полуорки, драконы и другие. Какая раса тебе ближе? 

(игрок может выбрать одну из предложенных рас или предложить свой вариант)

Отлично выбор! И наконец, определимся со специализацией. У нас есть воины, маги, воры, жрецы и другие классы. Какая специализация подходит твоему персонажу?

(игрок выбирает специализацию персонажа)

Прекрасно! Теперь дай мне немного описания твоего персонажа - его внешности, характера и целей. Учти, что это будет влиять на развитие сюжета и реакции других персонажей.

(игрок предоставляет описание персонажа)

Понял, спасибо за информацию об [Имя персонажа]. Ты готов начать свое приключение?

(есл